In [1]:
#creating an sqlite database table
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    create_connection("pythonsqlite.db")


2.6.0


In [2]:
#importing pandas and reading the BootcampSchedule sheet
import pandas as pd
BootcampSchedule = pd.read_excel('C:/Users/HP/Desktop/pythonacademy/BootcampSchedule.xlsx')

In [3]:
#Creating a database table named BootcampSchedule
connection = sqlite3.connect('pythonsqlite.db')
cursor = connection.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS BootcampSchedule (
              Month TEXT, 
              Date TEXT, 
              BootcampTime TEXT,
              Cohort TEXT,
              Program TEXT,
              Topic TEXT
              );''')

connection.commit()

In [4]:
# Loading the BootcampSchedule table with data from the BootcampSchedule sheet
BootcampSchedule.to_sql('BootcampSchedule', connection, if_exists='append', index=False)

28

In [5]:
# importing the ipywidgets for creating interacting interface
!!pip install ipython
from ipywidgets import *
from IPython.display import display

In [6]:
import datetime
from datetime import time


# Function to get current userTime from computer time
def currUserTime():
    now = datetime.datetime.now()
    current_time = now.strftime("%H:%M:%S") # formating time to hours minutes and secs
    return current_time
print(currUserTime())

14:29:16


In [7]:
#reading country and timezone from timezones_Detailed_588_Countries.csv

countryTime =pd.read_csv('timezones_Detailed_588_Countries.csv')
countries = list(countryTime['timezone'])
countryList =[]
for items in countries:
    if '/' in items:
        country=items.split('/')
        countryList.append(country[1])
    else:
        countryList.append(items)
        
countryTimeList = list(countryTime['offset'])
#Converting offset to hours
timezone=[]
timezoneList =[]
for item in countryTimeList:
    offsetHours = (item/(60*60))
    timezone.append(offsetHours)



In [8]:

countryAndTimeDict = dict(zip(countryList,timezone))
                          
def userTimezone(country):
    timezone = countryAndTimeDict[country] 
    if timezone >0:
        return f'The timezone of {country} is UTC + {timezone}'
    else:
        return f'The timezone of {country} is UTC {timezone}'


country = input('Please enter your country or city name: ')
userTimezone(country)

Please enter your country or city name: Lagos


'The timezone of Lagos is UTC + 1.0'

In [9]:
estOffset = -18000
 #A function to convert a 12-hour time and "am" or "pm" to a 24-hour value.    
def to_24hour(bootCampTime):
    estTime = bootCampTime.split(' ')
    if '.' in estTime[0]:
        est = estTime[0].split('.')
        estTimeHour = int(est[0])
    elif ':' in estTime[0]:
        est = estTime[0].split(':')
        estTimeHour = int(est[0])
    if estTime[1] == 'AM':
        return f'{00}.{est[1]}' if estTimeHour == 12 else f'{estTimeHour}.{est[1]}'
    else:
        return f'{12}.{est[1]}' if estTimeHour == 12 else f'{estTimeHour + 12}.{est[1]}'
    
bootCampTime = input('Please enter bootcamp time in format- xx.xx AM: ')
print(to_24hour(bootCampTime))

#A function to convert a 24-hour clock value to a 12-hour one.
def to_12hour(hour):
    hour = str(hour)
    if '.' in hour:
        est = hour.split('.')
        estTimeHour = int(est[0])
    elif ':' in hour:
        est = hour.split(':')
        estTimeHour = int(est[0])
    if int(estTimeHour) == 0:
        return (f'{12}.{est[1]} AM')
    elif int(estTimeHour) < 12:
        return (f'{estTimeHour}.{est[1]} AM')
    elif int(estTimeHour) == 12:
        return (f'{12}.{est[1]} PM')
    elif (int(estTimeHour)>12) & (int(estTimeHour)<24):
        return (f'{int(estTimeHour) - 12}.{est[1]} PM')
    else:
        return ('wrong format, enter a number between 0 and 24')

hour = input('Please enter bootcamp time in 24hr format- xx.xx: ')
to_12hour(hour)

Please enter bootcamp time in format- xx.xx AM: 2.00 PM 
14.00
Please enter bootcamp time in 24hr format- xx.xx: 21.00


'9.00 PM'

In [10]:
#function to convert bootcamptime from EST to user's time
def timeConverter(userCountry, bootCampTime):
    estHour = to_24hour(bootCampTime)
    countryOffset = countryAndTimeDict[userCountry]
    utcTime = to_12hour((float(estHour) - (estOffset/(60*60)) + 24)%24)
    utcTime24h = to_24hour(utcTime)
    userTime24h = ((float(utcTime24h) + countryOffset)+24)%24
    userTime = to_12hour(userTime24h)
    return userCountry,userTime

    
userCountry= input('Please enter your country or city name: ')
bootCampTime = input('Please enter bootcamp time format xx.xx AM: ')

country,time = timeConverter(userCountry,bootCampTime)
print(f'The Bootcamp time in {country} is {time}')



Please enter your country or city name: Lagos
Please enter bootcamp time format xx.xx AM: 6.00 AM
The Bootcamp time in Lagos is 12.0 PM


In [11]:
#Using an Interactive Interface
    
userCountry= input('Please enter your country or city name: ')
style = {'description_width': '150px'}
timeDropDown=widgets.Dropdown(options=['12.00 AM','1.00 AM','2.00 AM','3.00 AM','4.00 AM','5.00 AM','6.00 AM','7.00 AM',
                                      '8.00 AM','9.00 AM','10.00 AM','11.00 AM','12.00 PM','1.00 PM','2.00 PM','3.00 PM','4.00 PM','5.00 PM','6.00 PM','7.00 PM',
                                      '8.00 PM','9.00 PM','10.00 PM','11.00 PM'], value = '12.00 AM', description ='Select Bootcamp Time', disabled = False, style=style)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        bootCampTime = change['new']
        country,time = timeConverter(userCountry,bootCampTime)
        print(f'The Bootcamp time in {country} is {time}')

display(timeDropDown)
timeDropDown.observe(on_change)

Please enter your country or city name: Lagos


Dropdown(description='Select Bootcamp Time', options=('12.00 AM', '1.00 AM', '2.00 AM', '3.00 AM', '4.00 AM', …

The Bootcamp time in Lagos is 1.0 PM


In [12]:
# Reading columns from the database and converting to list
month = list(pd.read_sql("SELECT Month FROM BootcampSchedule", connection)['Month'])
date = list(pd.read_sql("SELECT   STRFTIME('%d/%m/%Y', Date)AS Date FROM BootcampSchedule", connection)['Date'])
time = list(pd.read_sql("SELECT BootcampTime FROM BootcampSchedule", connection)['BootcampTime'])
cohort= list(pd.read_sql("SELECT Cohort FROM BootcampSchedule", connection)['Cohort'])
program = list(pd.read_sql("SELECT Program FROM BootcampSchedule", connection)['Program'])
topic = list(pd.read_sql("SELECT Topic FROM BootcampSchedule", connection)['Topic'])

#accept user input
userCountry = input('Enter your City: ')
userTimeList = []
for item in time:
    userTime = timeConverter(userCountry,item)[1]
    userTimeList.append(userTime)
    
#printing new bootcamp schedule based on user country    
print('Below is the Bootcamp Schedule in your Country: ') 
for m, dt, t, chrt, pgram, tp in zip(month,date,userTimeList,cohort,program,topic):
    print(m, dt, t, chrt, pgram, tp, "\n")

Enter your City: Lagos
Below is the Bootcamp Schedule in your Country: 
July 07/07/2022 12.0 PM Cohort 14 Data Analytics Python Variables 

July 08/07/2022 4.0 PM Cohort 15 Data Science/ML Data Cleaning 

July 07/07/2022 12.0 PM Cohort 14 Data Science/ML Python Variables 

July 08/07/2022 4.0 PM Cohort 15 Data Analytics Data Visualization Project 1 

July 21/07/2022 12.0 PM Cohort 14 Data Analytics Python Container 

July 21/07/2022 4.0 PM Cohort 15 Data Science/ML Data Transformation 

July 22/07/2022 12.0 PM Cohort 14 Data Science/ML Python Container 

July 22/07/2022 4.0 PM Cohort 15 Data Analytics Data Visualization Project 2 

August 04/08/2022 12.0 PM Cohort 14 Data Analytics Python Container II 

August 04/08/2022 4.0 PM Cohort 15 Data Science/ML Data Transformation II 

August 05/08/2022 12.0 PM Cohort 14 Data Science/ML Python Container II 

August 05/08/2022 4.0 PM Cohort 15 Data Analytics Data Visualization Project 3 

August 18/08/2022 12.0 PM Cohort 14 Data Analytics Pytho

In [ ]:
currUserTime()
userTimeList